<a href="https://colab.research.google.com/github/cpython-projects/da_1709/blob/main/lesson_17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/cpython-projects/da_1305/refs/heads/main/real_estate_data.csv')
df.head()

,Unnamed: 0,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,kitchen_area,balcony,locality_name,days_exposition
0,0,7312500.0,108.0,2024-05-15,3,2.70,16.0,51.0,8,NaN,False,25.0,NaN,Kyiv,NaN
1,1,1884375.0,40.4,2024-08-14,1,NaN,11.0,18.6,1,NaN,False,11.0,2.0,Brovary,81.0
2,2,2922750.0,56.0,2023-11-06,2,NaN,5.0,34.3,4,NaN,False,8.3,0.0,Kyiv,558.0
3,3,36506250.0,159.0,2024-03-19,3,NaN,14.0,NaN,9,NaN,False,NaN,0.0,Kyiv,424.0
4,4,5625000.0,100.0,2024-06-12,2,3.03,14.0,32.0,13,NaN,False,41.0,NaN,Kyiv,121.0


# Групування даних з `groupby()`

## Що таке `groupby()`

Метод `groupby()` дозволяє згрупувати дані за одним або кількома критеріями для подальшого узагальнення (агрегації).

**Загальна формула:**

```python
df.groupby(ключ)[поля].агрегуюча_функція()
```

## Сценарії використання

In [3]:
df.columns.to_list()

['Unnamed: 0',
 'last_price',
 'total_area',
 'first_day_exposition',
 'rooms',
 'ceiling_height',
 'floors_total',
 'living_area',
 'floor',
 'is_apartment',
 'studio',
 'kitchen_area',
 'balcony',
 'locality_name',
 'days_exposition']

In [4]:
# Групування по одному полю — одна метрика
res = df.groupby('locality_name')['last_price'].mean()

In [5]:
type(res)

pandas.core.series.Series

In [6]:
res

,last_price
locality_name,
Borshchahivka,2.106405e+06
Boryspil,1.925962e+06
Boyarka,2.018181e+06
Brovary,1.966631e+06
Bucha,1.952773e+06
Hostomel,1.989404e+06
Irpin,2.023927e+06
Kyiv,4.535160e+06
Vyshneve,1.970079e+06


In [7]:
# Групування по одному полю — кілька метрик
df.groupby('locality_name')['last_price'].agg(['mean', 'median'])

,mean,median
locality_name,,
Borshchahivka,2.106405e+06,1856250.0
Boryspil,1.925962e+06,1771875.0
Boyarka,2.018181e+06,1771875.0
Brovary,1.966631e+06,1788750.0
Bucha,1.952773e+06,1771875.0
Hostomel,1.989404e+06,1800000.0
Irpin,2.023927e+06,1828125.0
Kyiv,4.535160e+06,3093750.0
Vyshneve,1.970079e+06,1757812.5


In [8]:
# Групування по кількох полях
df.groupby(['locality_name', 'rooms'])['last_price'].mean()

locality_name  rooms
Borshchahivka  0        1.228750e+06
               1        1.604200e+06
               2        2.008376e+06
               3        2.590366e+06
               4        3.818848e+06
                            ...     
Vyshneve       2        2.042682e+06
               3        2.492960e+06
               4        2.757305e+06
               5        5.017500e+06
               6        5.793750e+06
Name: last_price, Length: 72, dtype: float64

In [9]:
# Агрування кількох полів — кілька метрик
df.groupby(['rooms', 'locality_name'])['last_price'].agg(['mean', 'median'])

mean       median
rooms locality_name                           
0     Borshchahivka  1.228750e+06    1237500.0
      Boryspil       1.229414e+06    1276875.0
      Boyarka        1.195734e+06    1181250.0
      Brovary        1.256538e+06    1237500.0
      Bucha          1.422614e+06    1451250.0
...                           ...          ...
12    Kyiv           2.362500e+08  236250000.0
14    Kyiv           1.509159e+07   15091594.0
15    Kyiv           3.656250e+07   36562500.0
16    Kyiv           1.603125e+07   16031250.0
19    Kyiv           1.814062e+07   18140625.0

[72 rows x 2 columns]

## `reset_index()`

Після `groupby()` індексом стає групувальне поле (або поля). Щоб зробити з них **звичайні колонки**, потрібно:

```python
.reset_index()
```

Інакше графіки чи подальша робота будуть ускладнені.

In [13]:
res = df.groupby('locality_name')['last_price'].agg(['mean', 'median']).reset_index()
res[res.locality_name == 'Kyiv']

,locality_name,mean,median
7,Kyiv,4.535160e+06,3093750.0


## Задача: обчислити:

* середню ціну (`last_price`)
* кількість оголошень (`count`)
* медіанну площу (`total_area`)
  по кожному місту

In [15]:
df.groupby('locality_name')[['last_price', 'total_area']].agg(['mean', 'count', 'median']).reset_index()

locality_name    last_price                   total_area              
                          mean  count     median       mean  count median
0  Borshchahivka  2.106405e+06    966  1856250.0  53.359979    966  48.50
1       Boryspil  1.925962e+06    988  1771875.0  50.460273    988  45.15
2        Boyarka  2.018181e+06   1003  1771875.0  51.455354   1003  46.90
3        Brovary  1.966631e+06   1003  1788750.0  51.818594   1003  47.00
4          Bucha  1.952773e+06   1005  1771875.0  50.580995   1005  45.60
5       Hostomel  1.989404e+06   1026  1800000.0  51.668655   1026  46.60
6          Irpin  2.023927e+06    993  1828125.0  52.665116    993  48.00
7           Kyiv  4.535160e+06  15721  3093750.0  64.751488  15721  55.00
8       Vyshneve  1.970079e+06    994  1757812.5  51.426429    994  47.65

In [17]:
df.groupby('locality_name').agg({
    'last_price': ['mean', 'count'],
    'total_area': 'median'
}).reset_index()

locality_name    last_price        total_area
                          mean  count     median
0  Borshchahivka  2.106405e+06    966      48.50
1       Boryspil  1.925962e+06    988      45.15
2        Boyarka  2.018181e+06   1003      46.90
3        Brovary  1.966631e+06   1003      47.00
4          Bucha  1.952773e+06   1005      45.60
5       Hostomel  1.989404e+06   1026      46.60
6          Irpin  2.023927e+06    993      48.00
7           Kyiv  4.535160e+06  15721      55.00
8       Vyshneve  1.970079e+06    994      47.65

In [22]:
# Візуалізація: Топ-5 міст за кількістю оголошень

res = df.groupby('locality_name').agg(
    num_properties=('last_price', 'count')
).reset_index()
res.sort_values('num_properties', ascending=False, inplace=True)
res.head()


,locality_name,num_properties
7,Kyiv,15721
5,Hostomel,1026
4,Bucha,1005
3,Brovary,1003
2,Boyarka,1003


In [23]:
fig = px.bar(
    res[:5],
    x='locality_name',
    y='num_properties',
    text='num_properties'
)
fig.show()

# Агрегації `.agg()` в Pandas

## Навіщо потрібні агрегації?

Агрегація — це зведення (узагальнення) числових значень:

| Приклади запитань                      | Агрегація  |
| -------------------------------------- | ---------- |
| Яка середня площа квартир у Києві?     | `mean()`   |
| Скільки квартир виставлено у Броварах? | `count()`  |
| Яка максимальна ціна у студій?         | `max()`    |
| Яка медіанна кухня у 3-кімнатних?      | `median()` |

---

## Синтаксис `.agg()` — на одному або кількох полях

```python
df.agg({'col1': 'mean', 'col2': 'max'})
```

Або на одній колонці:

```python
df['last_price'].agg(['mean', 'median', 'std'])
```

---

## Комбінування з `groupby()`

```python
df.groupby('locality_name').agg({
    'last_price': ['mean', 'median', 'min', 'max'],
    'total_area': 'mean'
})
```

У результаті — **ієрархічний заголовок колонок (MultiIndex)** → бажано `.reset_index()` та перейменувати колонки.

In [27]:
summary = df[['last_price', 'total_area', 'kitchen_area']].agg(
    ['mean', 'median', 'std', 'min', 'max', 'count']
).round(1).reset_index()

summary


,index,last_price,total_area,kitchen_area
0,mean,3679621.2,60.3,10.6
1,median,2615625.0,52.0,9.1
2,std,6123945.0,35.7,5.9
3,min,6856.0,12.0,1.3
4,max,429187500.0,900.0,112.0
5,count,23699.0,23699.0,21421.0


In [28]:
# T - transpose

summary = df[['last_price', 'total_area', 'kitchen_area']].agg(
    ['mean', 'median', 'std', 'min', 'max', 'count']
).T.round(1)

summary

,mean,median,std,min,max,count
last_price,3679621.2,2615625.0,6123945.0,6856.0,429187500.0,23699.0
total_area,60.3,52.0,35.7,12.0,900.0,23699.0
kitchen_area,10.6,9.1,5.9,1.3,112.0,21421.0


In [30]:
# Візуалізація: Boxplot — ціна квартир по кількості кімнат

fig = px.box(
    df[df.rooms <= 5],
    x='rooms',
    y='last_price',
    color='rooms',
    title='Ціна квартир по кількості кімнат'
)
fig.show()

In [34]:
# Візуалізація: Histogram — площа квартир
fig = px.histogram(
    df[df.total_area > 300],
    x='total_area',
    nbins=50,
    title='Гистограма площа квартир'
)
fig.show()


## Нюанси

| Питання                       | Коментар                                     |
| ----------------------------- | -------------------------------------------- |
| `NaN`                         | Ігноруються у функціях агрегування           |
| `.agg()` після `groupby()`    | Повертає MultiIndex → `.reset_index()`       |
| `std`, `min`, `max`, `median` | Обчислюються лише для числових               |
| Для bool                      | Можна використовувати `mean()` — доля `True` |

---

# Зведені таблиці (`pivot_table`)

## Що таке `pivot_table`

`pivot_table()` — це розширений варіант `groupby`, який дозволяє:

* Зробити зведену таблицю по кількох категоріях
* Розмістити одну категорію по рядках, іншу по стовпцях
* Одразу застосувати агрегуючу функцію
* Аналог Excel PivotTable!

---

## Синтаксис

```python
pd.pivot_table(
    data=df,
    values='метрика',
    index='рядки',
    columns='стовпці',
    aggfunc='функція'
)
```

In [40]:
# Приклад: середня ціна по містах і кількості кімнат
df.groupby(['locality_name', 'rooms'])['last_price'].mean()

locality_name  rooms
Borshchahivka  0        1.228750e+06
               1        1.604200e+06
               2        2.008376e+06
               3        2.590366e+06
               4        3.818848e+06
                            ...     
Vyshneve       2        2.042682e+06
               3        2.492960e+06
               4        2.757305e+06
               5        5.017500e+06
               6        5.793750e+06
Name: last_price, Length: 72, dtype: float64

In [47]:
res = pd.pivot_table(
    data=df[df.rooms <= 4],
    values='last_price',
    index='locality_name',
    columns='rooms',
    aggfunc='mean',
)

In [48]:
# Візуалізація: Heatmap зведеної таблиці
fig = px.imshow(
    res,
    title='Середня ціна квартир по містах і кількості кімнат',
    labels=dict(x='Кількість кімнат', y='Місто', color='Середня ціна')
)
fig.show()

## Додаткові параметри

| Параметр                     | Значення            |
| ---------------------------- | ------------------- |
| `fill_value`                 | чим заповнити `NaN` |
| `margins=True`               | додати підсумки     |
| `aggfunc=['mean', 'median']` | кілька функцій      |


# Частотні таблиці — `pd.crosstab()`

## Що таке `crosstab`

`pd.crosstab()` — це спосіб підрахунку **кількості випадків** для комбінацій категоріальних змінних.

## Синтаксис

```python
pd.crosstab(index=df['рядки'], columns=df['стовпці'])
```

In [50]:
df.columns.to_list()

['Unnamed: 0',
 'last_price',
 'total_area',
 'first_day_exposition',
 'rooms',
 'ceiling_height',
 'floors_total',
 'living_area',
 'floor',
 'is_apartment',
 'studio',
 'kitchen_area',
 'balcony',
 'locality_name',
 'days_exposition']

In [51]:
# Приклад: скільки квартир є студіями або ні по містах
pd.crosstab(index=df['locality_name'], columns=df['studio'])

studio,False,True
locality_name,,
Borshchahivka,960,6
Boryspil,983,5
Boyarka,994,9
Brovary,995,8
Bucha,994,11
Hostomel,1014,12
Irpin,988,5
Kyiv,15635,86
Vyshneve,987,7


In [52]:
# Приклад: студії × відкритий план
pd.crosstab(index=df['studio'], columns=df['is_apartment'])

is_apartment,False,True
studio,,
False,2710,50
True,15,0


In [53]:
df.shape

(23699, 15)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  kitchen_area          21421 non-null  float64
 12  balcony               12180 non-null  float64
 13  locality_name         23699 non-null  object 
 14  days_exposition       20518 non-null  float64
dtypes: bool(1), float64

## Нюанси

| Питання                            | Коментар                              |
| ---------------------------------- | ------------------------------------- |
| `NaN` у категоріях                 | не враховуються                       |
| `normalize='index'`                | показує **частки** замість кількостей |
| Можна додати `values=`, `aggfunc=` | як у pivot\_table                     |

---

In [55]:
# Частки типів квартир (кількість кімнат) у кожному місті
pd.crosstab(
    index=df['locality_name'],
    columns=df['rooms']
)

rooms,0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,19
locality_name,,,,,,,,,,,,,,,,,
Borshchahivka,9,362,327,228,32,5,1,2,0,0,0,0,0,0,0,0,0
Boryspil,8,398,369,187,23,2,1,0,0,0,0,0,0,0,0,0,0
Boyarka,13,411,331,220,24,3,0,0,1,0,0,0,0,0,0,0,0
Brovary,13,383,360,203,41,3,0,0,0,0,0,0,0,0,0,0,0
Bucha,11,406,345,210,28,4,1,0,0,0,0,0,0,0,0,0,0
Hostomel,17,398,362,222,20,7,0,0,0,0,0,0,0,0,0,0,0
Irpin,6,358,398,199,24,7,1,0,0,0,0,0,0,0,0,0,0
Kyiv,110,4937,5106,4123,968,291,99,57,11,8,3,2,1,2,1,1,1
Vyshneve,10,394,342,222,20,4,2,0,0,0,0,0,0,0,0,0,0


In [56]:
pd.crosstab(
    index=df['locality_name'],
    columns=df['rooms'],
    normalize='index'
)

rooms,0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,19
locality_name,,,,,,,,,,,,,,,,,
Borshchahivka,0.009317,0.374741,0.338509,0.236025,0.033126,0.005176,0.001035,0.002070,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Boryspil,0.008097,0.402834,0.373482,0.189271,0.023279,0.002024,0.001012,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Boyarka,0.012961,0.409771,0.330010,0.219342,0.023928,0.002991,0.000000,0.000000,0.000997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Brovary,0.012961,0.381854,0.358923,0.202393,0.040877,0.002991,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Bucha,0.010945,0.403980,0.343284,0.208955,0.027861,0.003980,0.000995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Hostomel,0.016569,0.387914,0.352827,0.216374,0.019493,0.006823,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Irpin,0.006042,0.360524,0.400806,0.200403,0.024169,0.007049,0.001007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Kyiv,0.006997,0.314039,0.324788,0.262261,0.061574,0.018510,0.006297,0.003626,0.000700,0.000509,0.000191,0.000127,0.000064,0.000127,0.000064,0.000064,0.000064
Vyshneve,0.010060,0.396378,0.344064,0.223340,0.020121,0.004024,0.002012,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [57]:
# Середня площа квартир по населених пунктах і кількості кімнат

pd.crosstab(
    index=df['locality_name'],
    columns=df['rooms'],
    values=df['total_area'],
    aggfunc='mean'
)

rooms,0,1,2,3,4,5,6,7,8,9,10,11,12,14,15,16,19
locality_name,,,,,,,,,,,,,,,,,
Borshchahivka,27.377778,36.297541,53.463578,70.368158,96.900000,115.360000,380.000000,287.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Boryspil,25.657500,36.911809,51.966802,71.817326,84.986957,135.000000,128.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Boyarka,24.115385,36.191776,53.485650,70.926136,100.662500,116.833333,NaN,NaN,347.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brovary,25.508462,36.670522,51.950500,73.245961,88.427317,133.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Bucha,26.032727,36.221527,51.699855,69.743143,91.646071,118.250000,320.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hostomel,25.224706,37.252915,53.116713,70.851306,101.716000,109.285714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Irpin,28.338333,36.821397,52.891181,73.133568,96.645833,118.328571,192.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Kyiv,32.059636,38.312064,57.517372,80.440825,110.912056,166.755292,200.664141,264.538596,251.209091,305.975,259.566667,188.9,900.0,304.2,590.0,270.0,374.6
Vyshneve,25.957000,36.332716,54.462135,70.228198,80.050000,112.600000,137.550000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Загальна ціна (last\_price) по типу квартири (студія, не студія) і наявності відкритого планування


# `resample()` — агрегування за датами

## Що таке `resample`

`resample()` — це аналог `groupby()` **по часу**, який працює з колонкою `datetime`.

Необхідно:

* мати колонку типу `datetime64[ns]`
* встановити її як **індекс**

---

## Синтаксис:

```python
df.resample('період')['метрика'].agg(функція)
```


### Базові часові одиниці

| Частота     | Alias             | Опис             |
| ----------- | ----------------- | ---------------- |
| Day         | `'D'`             | календарний день |
| Hour        | `'H'` або `'h'`   | година           |
| Minute      | `'T'` або `'min'` | хвилина          |
| Second      | `'S'` або `'s'`   | секунда          |
| Millisecond | `'ms'`            | мілісекунда      |
| Microsecond | `'us'`            | мікросекунда     |
| Nanosecond  | `'ns'`            | наносекунда      |

---

### Бізнес-час і бізнес-дні

| Частота                  | Alias   | Опис                                                 |
| ------------------------ | ------- | ---------------------------------------------------- |
| BDay / BusinessDay       | `'B'`   | бізнес-день (пн–пт)                                  |
| CDay / CustomBusinessDay | `'C'`   | кастомний бізнес-день (за користувацьким календарем) |
| BusinessHour             | `'bh'`  | бізнес-година                                        |
| CustomBusinessHour       | `'cbh'` | кастомна бізнес-година                               |

---

### Тижні

| Частота         | Alias    | Опис                                           |
| --------------- | -------- | ---------------------------------------------- |
| Week            | `'W'`    | тиждень, можна якір на день (`W-MON`, `W-FRI`) |
**Приклади**

* `'W'` — тиждень, кінець у неділю
* `'W-MON'` — тиждень з кінцем у понеділок

---

### Місячні частоти

| Частота        | Alias              | Опис                            |
| -------------- | ------------------ | ------------------------------- |
| MonthEnd       | `'M'` або `'ME'`   | кінець календарного місяця      |
| MonthBegin     | `'MS'`             | початок місяця                  |
| BMonthEnd      | `'BM'` або `'BME'` | бізнес-кінець місяця            |
| BMonthBegin    | `'BMS'`            | бізнес-початок місяця           |

---

### Квартали

| Частота       | Alias              | Опис                                    |
| ------------- | ------------------ | --------------------------------------- |
| QuarterEnd    | `'Q'` або `'QE'`   | кінець календарного кварталу            |
| QuarterBegin  | `'QS'`             | початок кварталу                        |
| BQuarterEnd   | `'BQ'` або `'BQE'` | бізнес-кінець кварталу                  |
| BQuarterBegin | `'BQS'`            | бізнес-початок кварталу                 |

**Приклади**

* `'Q-MAR'` — квартал, що закінчується у березні
* `'QS-APR'` — квартал починається у квітні

---

### Річні частоти

| Частота    | Alias                   | Опис                       |
| ---------- | ----------------------- | -------------------------- |
| YearEnd    | `'A'` або `'YE'`        | кінець календарного року   |
| YearBegin  | `'AS'`, `'YS'`, `'BYS'` | початок року               |
| BYearEnd   | `'BA'`, `'BYE'`         | бізнес-кінець року         |
| BYearBegin | `'BAS'`, `'BYS'`        | бізнес-початок року        |


**Приклади**

* `'A-DEC'` — рік закінчується у грудні
* `'AS-JUL'` — рік починається у липні

In [58]:
# Підготовка даних
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  kitchen_area          21421 non-null  float64
 12  balcony               12180 non-null  float64
 13  locality_name         23699 non-null  object 
 14  days_exposition       20518 non-null  float64
dtypes: bool(1), float64

In [59]:
df.first_day_exposition.unique()

array(['2024-05-15', '2024-08-14', '2023-11-06', ..., '2021-08-26',
       '2022-08-14', '2022-09-19'], dtype=object)

In [60]:
df['first_day_exposition'] = pd.to_datetime(df['first_day_exposition'], format='%Y-%m-%d', errors='coerce')

In [62]:
df.set_index('first_day_exposition', inplace=True)

In [64]:
df.sort_index(inplace=True)

In [65]:
df

,Unnamed: 0,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,kitchen_area,balcony,locality_name,days_exposition
first_day_exposition,,,,,,,,,,,,,,
2021-01-18,18843,1968750.0,48.2,2,2.50,5.0,27.40,2,NaN,False,7.7,3.0,Boryspil,1580.0
2021-01-26,1109,19619188.0,95.8,2,NaN,6.0,58.30,5,True,False,20.0,1.0,Kyiv,1572.0
2021-02-14,9553,5962500.0,80.0,3,2.85,17.0,44.00,9,NaN,False,15.0,NaN,Kyiv,1553.0
2021-03-26,1885,7481250.0,79.6,2,3.00,8.0,42.70,7,NaN,False,18.0,NaN,Kyiv,1513.0
2021-03-27,20969,7481250.0,133.0,4,3.00,5.0,58.00,2,NaN,False,45.0,NaN,Kyiv,1512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-30,22052,2475000.0,49.1,4,NaN,5.0,36.90,1,NaN,False,5.8,NaN,Kyiv,NaN
2024-12-30,10574,1350000.0,31.7,1,NaN,5.0,18.00,4,NaN,False,6.0,NaN,Boryspil,NaN
2024-12-30,20875,5625000.0,102.4,4,3.00,7.0,65.00,6,NaN,False,10.0,NaN,Kyiv,NaN


In [66]:
# Приклад: середня ціна квартир по місяцях
res = df.resample('M')['last_price'].mean()
res

,last_price
first_day_exposition,
2021-01-31,1.079397e+07
2021-02-28,5.962500e+06
2021-03-31,5.812500e+06
2021-04-30,5.086125e+06
2021-05-31,3.963750e+06
2021-06-30,8.039714e+06
2021-07-31,6.687563e+06
2021-08-31,5.806276e+06
2021-09-30,5.432349e+06


In [67]:
df.resample('W')['last_price'].mean()

,last_price
first_day_exposition,
2021-01-24,1.968750e+06
2021-01-31,1.961919e+07
2021-02-07,NaN
2021-02-14,5.962500e+06
2021-02-21,NaN
...,...
2024-12-08,3.872349e+06
2024-12-15,3.819595e+06
2024-12-22,3.672118e+06


In [68]:
df.resample('Q')['last_price'].mean()

,last_price
first_day_exposition,
2021-03-31,7.497990e+06
2021-06-30,5.854055e+06
2021-09-30,5.817554e+06
2021-12-31,4.984287e+06
2022-03-31,4.867995e+06
2022-06-30,5.930630e+06
2022-09-30,4.504193e+06
2022-12-31,5.419975e+06
2023-03-31,4.206238e+06


In [ ]:
res = df.resample('M')['last_price'].mean()

In [69]:
# Візуалізація: динаміка середньої ціни по місяцях
fig = px.line(
    res,
    title='Середня ціна квартир по місяцях'
)
fig.show()

In [70]:
# Кількість оголошень по тижнях
res = df.resample('W')['last_price'].count()
res

,last_price
first_day_exposition,
2021-01-24,1
2021-01-31,1
2021-02-07,0
2021-02-14,1
2021-02-21,0
...,...
2024-12-08,134
2024-12-15,108
2024-12-22,84


In [71]:
# Візуалізація: кількість оголошень по тижнях
fig = px.bar(
    res,
    title='Кількість оголошень по тижнях'
)
fig.show()

In [72]:
fig = px.line(
    res,
    title='Кількість оголошень по тижнях'
)
fig.show()

## Навіщо аналітику `resample`

| Що дізнаємось                         | Як це корисно                       |
| ------------------------------------- | ----------------------------------- |
| Сезонність ринку                      | порівняння цін по місяцях/роках     |
| Активність користувачів               | по кількості оголошень              |
| Аналіз впливу подій (інфляція, війна) | тренди по кварталах                 |
| Маркетинг                             | коли зростає інтерес до нерухомості |

---

## Нюанси

| Проблема          | Рішення                                 |
| ----------------- | --------------------------------------- |
| Не дата в індексі | `.set_index('datetime')`                |
| Пропущені періоди | Автоматично створюються `NaN`           |
| Смугастий графік  | Використовуйте `'M'`, `'Q'`, а не `'D'` |
| Великі значення   | Обрізати викиди перед побудовою         |

---

# `pd.cut()` — групування числових значень у категорії

## Що таке `cut`

`cut()` розбиває числову колонку на **інтервали (бінінг)**. Наприклад, можна створити категорії квартир:

* до 40 м² → «мала»
* 40–80 м² → «середня»
* понад 80 м² → «велика»

Це корисно для:

* **аналізу розподілу** (гістограми, pie charts)
* **порівнянь середніх цін між групами**
* **створення сегментів**

---

## Синтаксис

```python
pd.cut(Series, bins, labels=..., include_lowest=True)
```

| Параметр         | Значення                         |
| ---------------- | -------------------------------- |
| `bins`           | кількість або список меж         |
| `labels`         | імена категорій                  |
| `include_lowest` | включити нижню межу в перший бін |

---

In [74]:
# Приклад: Розбиття квартир за площею
df['area_category'] = pd.cut(
    df['total_area'],
    bins=[0, 40, 80, df.total_area.max()],
    labels=['мала', 'середня', 'велика'],
    include_lowest=True
)

In [75]:
df

,Unnamed: 0,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,kitchen_area,balcony,locality_name,days_exposition,area_category
first_day_exposition,,,,,,,,,,,,,,,
2021-01-18,18843,1968750.0,48.2,2,2.50,5.0,27.40,2,NaN,False,7.7,3.0,Boryspil,1580.0,середня
2021-01-26,1109,19619188.0,95.8,2,NaN,6.0,58.30,5,True,False,20.0,1.0,Kyiv,1572.0,велика
2021-02-14,9553,5962500.0,80.0,3,2.85,17.0,44.00,9,NaN,False,15.0,NaN,Kyiv,1553.0,середня
2021-03-26,1885,7481250.0,79.6,2,3.00,8.0,42.70,7,NaN,False,18.0,NaN,Kyiv,1513.0,середня
2021-03-27,20969,7481250.0,133.0,4,3.00,5.0,58.00,2,NaN,False,45.0,NaN,Kyiv,1512.0,велика
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-30,22052,2475000.0,49.1,4,NaN,5.0,36.90,1,NaN,False,5.8,NaN,Kyiv,NaN,середня
2024-12-30,10574,1350000.0,31.7,1,NaN,5.0,18.00,4,NaN,False,6.0,NaN,Boryspil,NaN,мала
2024-12-30,20875,5625000.0,102.4,4,3.00,7.0,65.00,6,NaN,False,10.0,NaN,Kyiv,NaN,велика


In [76]:
df.area_category.value_counts()

,count
area_category,
середня,13912
мала,6042
велика,3745


In [ ]:
# Візуалізація: Середня ціна за розміром квартири


In [ ]:
# Приклад: Розбиття цін на групи



# Аналіз середньої площі в кожній ціновій категорії




In [77]:
# Альтернатива: qcut() — автоматичне розбиття на квантилі
# Корисно, коли хочемо однакову кількість спостережень у групі, а не однакову ширину інтервалу.

df['area_auto_category'] = pd.qcut(
    df['total_area'],
    q=3,
    labels=['мала', 'середня', 'велика'],
)
df

,Unnamed: 0,last_price,total_area,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,kitchen_area,balcony,locality_name,days_exposition,area_category,area_auto_category
first_day_exposition,,,,,,,,,,,,,,,,
2021-01-18,18843,1968750.0,48.2,2,2.50,5.0,27.40,2,NaN,False,7.7,3.0,Boryspil,1580.0,середня,середня
2021-01-26,1109,19619188.0,95.8,2,NaN,6.0,58.30,5,True,False,20.0,1.0,Kyiv,1572.0,велика,велика
2021-02-14,9553,5962500.0,80.0,3,2.85,17.0,44.00,9,NaN,False,15.0,NaN,Kyiv,1553.0,середня,велика
2021-03-26,1885,7481250.0,79.6,2,3.00,8.0,42.70,7,NaN,False,18.0,NaN,Kyiv,1513.0,середня,велика
2021-03-27,20969,7481250.0,133.0,4,3.00,5.0,58.00,2,NaN,False,45.0,NaN,Kyiv,1512.0,велика,велика
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-30,22052,2475000.0,49.1,4,NaN,5.0,36.90,1,NaN,False,5.8,NaN,Kyiv,NaN,середня,середня
2024-12-30,10574,1350000.0,31.7,1,NaN,5.0,18.00,4,NaN,False,6.0,NaN,Boryspil,NaN,мала,мала
2024-12-30,20875,5625000.0,102.4,4,3.00,7.0,65.00,6,NaN,False,10.0,NaN,Kyiv,NaN,велика,велика


In [ ]:
# Візуалізація: розподіл площі по цінових квантилях


## Навіщо аналітику `cut` і `qcut`

| Задача                             | Рішення                |
| ---------------------------------- | ---------------------- |
| Побудова сегментів                 | `cut` або `qcut`       |
| Когортний аналіз (покоління, ціна) | `cut` по даті або ціні |
| Спрощення числових фіч             | категоризація          |
| Візуалізація (pie, bar, boxplot)   | категорії зручніші     |

---

## Типові помилки

| Проблема                          | Рішення                                   |
| --------------------------------- | ----------------------------------------- |
| Значення на межі не включене      | `include_lowest=True`                     |
| Некоректні категорії              | задати `labels` вручну або перевірити     |
| Багато унікальних значень у `cut` | замість `cut` — використовувати `qcut`    |
| Нерівномірні категорії            | вручну задати `bins`, якщо треба контроль |

---